# Setup Notebook and Repository


In [1]:
import os
import shutil

# Constants
SRC_FOLDER = "/kaggle/input/repository8"
DST_FOLDER = "/kaggle/working"
DIRS_TO_CREATE = [
    "/kaggle/working/logs",
    "/kaggle/working/data/cache",
    "/kaggle/working/checkpoints/other_models",
    "/kaggle/working/checkpoints/best_models",
]


def copy_all_python_files(src, dst):
    for root, _, files in os.walk(src):
        dest_path = os.path.join(dst, os.path.relpath(root, src))
        os.makedirs(dest_path, exist_ok=True)
        print(f"Created/checked directory: {dest_path}")
        for file in filter(lambda f: f.endswith(".py"), files):
            src_file = os.path.join(root, file)
            dst_file = os.path.join(dest_path, file)
            shutil.copyfile(src_file, dst_file)
            print(f"Copied {src_file} to {dst_file}")


def create_directories(dirs):
    for directory in dirs:
        os.makedirs(directory, exist_ok=True)
        print(f"Created/checked directory: {directory}")


# Execution
copy_all_python_files(SRC_FOLDER, DST_FOLDER)
create_directories(DIRS_TO_CREATE)

Created/checked directory: /kaggle/working/.
Copied /kaggle/input/repository8/generics.py to /kaggle/working/./generics.py
Copied /kaggle/input/repository8/main.py to /kaggle/working/./main.py
Created/checked directory: /kaggle/working/utils
Copied /kaggle/input/repository8/utils/inference_utils.py to /kaggle/working/utils/inference_utils.py
Copied /kaggle/input/repository8/utils/failure_utils.py to /kaggle/working/utils/failure_utils.py
Copied /kaggle/input/repository8/utils/mspca_utils.py to /kaggle/working/utils/mspca_utils.py
Copied /kaggle/input/repository8/utils/loader_utils.py to /kaggle/working/utils/loader_utils.py
Copied /kaggle/input/repository8/utils/ica_utils.py to /kaggle/working/utils/ica_utils.py
Copied /kaggle/input/repository8/utils/grid_search_utils.py to /kaggle/working/utils/grid_search_utils.py
Copied /kaggle/input/repository8/utils/signal_preprocessing_utils.py to /kaggle/working/utils/signal_preprocessing_utils.py
Copied /kaggle/input/repository8/utils/eeg_proce

In [2]:
import sys

cache_folder_name = "config-normalize-group-raw-ica-0"


class Paths:
    PRE_LOADED_EEGS = "/kaggle/input/brain-eeg-spectrograms/eeg_specs.npy"
    PRE_LOADED_SPECTROGRAMS = "/kaggle/input/brain-spectrograms/specs.npy"
    TRAIN_CSV = "/kaggle/input/hms-harmful-brain-activity-classification/train.csv"
    TEST_CSV = "/kaggle/input/hms-harmful-brain-activity-classification/test.csv"
    TEST_EEGS = "/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/"
    TRAIN_EEGS = "/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/"
    TRAIN_SPECTROGRAMS = (
        "/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/"
    )
    TEST_SPECTROGRAMS = (
        "/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/"
    )

    OTHER_MODEL_CHECKPOINTS = "/kaggle/working/checkpoints/other_models/"
    BEST_MODEL_CHECKPOINTS = "/kaggle/working/checkpoints/best_models/"
    CACHE_PATH_READ = f"/kaggle/input/{cache_folder_name}/"
    CACHE_PATH_WRITE = f"/kaggle/working/data/cache/"
    TENSORBOARD = "/kaggle/working/tensorboard/"
    TENSORBOARD_MODELS = "/kaggle/working/tensorboard/models/"
    TENSORBOARD_TRAINING = "/kaggle/working/tensorboard/training/"
    TENSORBOARD_DATASETS = "/kaggle/working/tensorboard/datasets/"
    TENSORBOARD_INFERENCE = "/kaggle/working/tensorboard/inference/"

    LOG_PATH = "/kaggle/working/logs/"


import generics

generics.Paths = Paths
sys.modules["generics"] = generics

In [3]:
import torch
from generics import Generics

Generics.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Training


In [4]:
from datasets.data_loader import CustomDataset
from datasets.data_loader_configs import (
    BaseDataConfig,
    ExtremelySmallBaseConfig,
    Config_Normalize_Group_Raw_ICA,
)
from generics import Paths
from models.CustomModel import CustomModel
from models.custom_model_configs import BaseModelConfig
from utils.inference_utils import perform_inference
from utils.loader_utils import load_main_dfs
from utils.training_utils import train


class EfficientNetB0Config_Big_Weight_Decay(BaseModelConfig):
    MODEL = "tf_efficientnet_b0"
    FREEZE = False
    EPOCHS = 20
    GRADIENT_ACCUMULATION_STEPS = 1
    WEIGHT_DECAY = 0.01
    MAX_LEARNING_RATE_SCHEDULERER = 0.001
    USE_KAGGLE_SPECTROGRAMS = False


data_loader_config = Config_Normalize_Group_Raw_ICA
model_config = EfficientNetB0Config_Big_Weight_Decay
train_df, val_df, _ = load_main_dfs(data_loader_config, train_val_split=(0.8, 0.2))
# Load datasets
train_dataset = CustomDataset(
    config=data_loader_config, main_df=train_df, mode="train", cache=True
)
val_dataset = CustomDataset(
    config=data_loader_config, main_df=val_df, mode="val", cache=True
)

# Print summaries
train_dataset.print_summary()
val_dataset.print_summary()

# Initialize and train the model
model = CustomModel(model_config)
print("starting training")
train(model=model, train_dataset=train_dataset, val_dataset=val_dataset)

2024-02-27 10:40:53.544263: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 10:40:53.544362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 10:40:53.696095: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 10:41:09,686 - data_loader.log - INFO - Loading dataset from cache: Config_Normalize_Group_Raw_ICA_0_train.npz


Train non-overlapp eeg_id shape: (17089, 12)


2024-02-27 10:41:24,100 - data_loader.log - INFO - Dataset loaded: train mode, 1560 samples, with config Config_Normalize_Group_Raw_ICA
2024-02-27 10:41:24,104 - data_loader.log - INFO - Loading dataset from cache: Config_Normalize_Group_Raw_ICA_0_val.npz
2024-02-27 10:41:24,104 - data_loader.log - INFO - Loading dataset from cache: Config_Normalize_Group_Raw_ICA_0_val.npz
2024-02-27 10:41:27,566 - data_loader.log - INFO - Dataset loaded: val mode, 390 samples, with config Config_Normalize_Group_Raw_ICA
2024-02-27 10:41:27,566 - data_loader.log - INFO - Dataset loaded: val mode, 390 samples, with config Config_Normalize_Group_Raw_ICA
2024-02-27 10:41:27,615 - models/models.CustomModel - INFO - Using device: cuda


Dataset Summary:
Mode: train
Total Samples: 1560
Unique Patients: 1560
Unique EEGs: 1560
Unique Spectrograms: 1560
Augmentation: Disabled
Label Distribution:
seizure_vote    496.451309
lpd_vote         61.605373
gpd_vote         63.876083
lrda_vote        92.379873
grda_vote       315.493842
other_vote      530.193520
dtype: float64

Vote Statistics:
      seizure_vote  lpd_vote  gpd_vote  lrda_vote  grda_vote  other_vote
mean      0.318238  0.039491  0.040946   0.059218   0.202240    0.339868
var       0.196658  0.021626  0.024108   0.036461   0.133245    0.170138
Spectrograms Loaded: 1560
EEG Spectrograms Loaded: 1560

Configuration Summary:
+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+
| Configuration                  | Value                                                                                                                                |
+-------

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

...

Validation:  96%|█████████▌| 23/24 [00:02<00:00,  9.52valid_batch/s]

1.22161189628684


Validation: 100%|██████████| 24/24 [00:02<00:00,  9.31valid_batch/s]
2024-02-27 11:25:13,483 - utils.training_utils - INFO - Average Training Loss: 0.0897
2024-02-27 11:25:13,484 - utils.training_utils - INFO - Average Validation Loss: 1.2040


1.2039699678619702
EVAL: [23/24] Elapsed 0m 2s (remain 0m 0s) Loss: 1.2040 


,seizure_vote_pred,lpd_vote_pred,gpd_vote_pred,lrda_vote_pred,grda_vote_pred,other_vote_pred,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,0.800000,0.0,0.00,0.000000,0.0,0.200000,0.766595,0.002131,0.001167,0.005958,0.015679,0.208470
1,1.000000,0.0,0.00,0.000000,0.0,0.000000,0.199931,0.061032,0.014396,0.190258,0.470848,0.063535
2,0.000000,0.0,0.00,0.000000,0.0,1.000000,0.015103,0.009286,0.002677,0.668199,0.171733,0.133002
3,0.866667,0.0,0.00,0.000000,0.0,0.133333,0.034470,0.002495,0.000883,0.082940,0.821042,0.058169
4,0.000000,0.0,0.00,0.466667,0.0,0.533333,0.020899,0.013780,0.001336,0.406173,0.365299,0.192513
...,...,...,...,...,...,...,...,...,...,...,...,...
379,1.000000,0.0,0.00,0.000000,0.0,0.000000,0.032828,0.016473,0.003000,0.567369,0.153988,0.226342
380,1.000000,0.0,0.00,0.000000,0.0,0.000000,0.155743,0.018285,0.009157,0.108522,0.524028,0.184264
381,1.000000,0.0,0.00,0.000000,0.0,0.000000,0.959643,0.000392,0.000144,0.000353,0.000127,0.039341
382,0.913043,0.0,0.00,0.043478,0.0,0.043478,0.863092,0.007334,0.001210,0.025518,0.067812,0.035034
